# Trabalho final automação industrial

## Funções de leitura do código de barras

In [24]:
def produto(codigo):
    """Identifica o componente a ser produzido pelo código de barras"""
    tipo = int(codigo[0])
    
    if tipo == 1:
        produto = 'Produto 1'
    elif tipo == 2:
        produto = 'Produto 2'
    else:
        raise Exception('Código inválido! Tipo de produto não cadastrado.')
        
    return produto

In [25]:
def quantidade(codigo):
    """Quantidade do produto a ser produzido"""
    try:
        qtd = int(codigo[1:5])
        return qtd
    except:
        raise Exception('Código inválido! Erro na leitura de quantidades.')

In [26]:
def leitura(codigo):
    """Função que realiza a leitura do código e retorna o 
    tipo do produto e quantidade que deve ser produzida"""
    tipo_produto = produto(codigo)
    qtd = quantidade(codigo)
    
    if tipo_produto == 'Produto 1':
        temperatura = '20 °C'  # Celsius
        pressao = '10 psi' # Psi
        velocidade = '10 cm/s'  # cm / s
    elif tipo_produto == 'Produto 2':
        temperatura = '15°C'
        pressao = '8 psi'
        velocidade = '12 cm/s'
    
    return tipo_produto, qtd, temperatura, pressao, velocidade

## Tela

In [27]:
def LimpaTela():
    print('\n' * 5)

In [28]:
def Menu():
    """Menu principal de opções do usuário"""
    opcao = 0
    while opcao != 4:
        LimpaTela()
        print('-'*30)
        print('       Tela inicial')
        print('-'*30)
        print('1 - Cadastro de ordem')
        print('-'*30)
        print('2 - Iniciar produção')
        print('-'*30)
        print('3 - Finalizar produção')
        print('-'*30)
        print('4 - Sair')
        print('-'*30)
        print('5 - Status das ordens')
        print('-'*30)
        print('6 - Relatórios')
        print('-'*30)
        try:
            opcao = int(input('Entre com a opção desejada: '))
        except:
            print('Opção inválida')
        
        # Mudança para o menu referente à opção
        if opcao == 1 or opcao == 2 or opcao == 3 or opcao == 5 or opcao == 6:
            print()
            TelaCadastro(opcao)

In [29]:
def TelaCadastro(opcao):
    """Tela de opções específicas chamadas através da função menu"""
    # Cadastro de ordem de produção
    if opcao == 1:
        print('-'*30)
        print('Cadastro de ordem de produção')
        print('-'*30)
        try:
            CadastroOrdem()
        except:
            print('Ordem inválida!')
    
    # Iniciar produção
    elif opcao == 2:
        # Seleciona em qual máquina será feita a produção
        print('-'*35)
        print('Iniciar a produção de uma ordem')
        print('-'*35)
        print('1 - Máquina A')
        print('2 - Máquina B')
        print('-'*30)
        try:
            maquina = int(input('Insira o código da máquina: '))
            print('-'*30)
            if maquina == 1: 
                ProduzirOrdem('A')
            elif maquina == 2:
                ProduzirOrdem('B')
        except:
            print('Entrada inválida!')
    
    # Finalizar produção
    elif opcao == 3:
        # Seleciona máquina para finalizar produção
        print('-'*35)
        print('Finalizar a produção de uma ordem')
        print('-'*35)
        try:
            FinalizarOrdem()
        except:
            print('Entrada inválida!')
    
    # Listagem de ordens cadastradas
    elif opcao == 5:
        print('-'*30)
        ListaOrdens()
        print('-'*30)
        
    # Emissão de relatórios
    elif opcao == 6:
        print('-'*35)
        print('      Emissão de relatórios')
        print('-'*35)
        print('1 - Taxa de produção em uma data')
        print('-'*35)
        print('2 - Tempo de leadtime em uma data')
        print('-'*35)
        try:
            escolha = int(input('Escolha a opção de relatório: '))
        except:
            print('Escolha inválida!')
            escolha = 3
        # Taxa de produção de uma máquina em uma data
        if escolha == 1:
            print('-'*35)
            print('1 - Máquina A')
            print('2 - Máquina B')
            print('-'*30)
            try:
                maquina = int(input('Insira o código da máquina: '))
                print('-'*30)
                if maquina == 1: 
                    TaxaProducao('A')
                elif maquina == 2:
                    TaxaProducao('B')
            except:
                print('Entrada inválida!')
        # Leadtime das ordens em uma data
        elif escolha == 2:
            LeadTime()

## Integração SQL

In [30]:
import mysql.connector

In [31]:
def ConectaBanco():
    """Ligação com banco de dados MySQL, inserir seu usuário e senha"""
    cnx = mysql.connector.connect(database='manufatura', host='localhost', user='root', passwd='urpassword')
    cnx.autocommit = True
    return cnx

### Ordens

In [32]:
def InsertOrdem(codigo, componente, qtd, status, temperatura, pressao, velocidade):
    """Cadastra uma ordem de produção no banco de dados"""
    execucao = True
    sql = f"INSERT INTO ordem VALUES ('{codigo}', '{componente}', {qtd}, '{status}', \
                                      '{temperatura}', '{pressao}', '{velocidade}')"
    
    try:
        cursor.execute(sql)
        print('Ordem cadastrada com sucesso!')
    except mysql.connector.Error as err:
        print(f"Erro: {err.errno} - {err.msg}")
        execucao = False
        
    return execucao

In [33]:
def CadastroOrdem():
    """Decodifica o código de barras para cadastro da ordem no banco de dados"""
    codigo = input('Leia o código para cadastro: ')
    dados = leitura(codigo)
    
    componente = dados[0]
    qtd = dados[1]
    temperatura = dados[2]
    pressao = dados[3]
    velocidade = dados[4]
    status = 'Em fila'
    
    InsertOrdem(codigo, componente, qtd, status, temperatura, pressao, velocidade)

In [34]:
def ListaOrdens():
    """Apresenta todas as ordens cadastradas e seus respectivos status"""
    cursor.execute("SELECT codigo, status FROM ordem")
    c = cursor.fetchall()

    print('Ordens Cadastradas \n')
    for ordem in c:
        codigo = ordem[0]
        status = ordem[1]
        print(f'Código: {codigo}   |   Status: {status}')

### Produção

In [35]:
def Status(codigo):
    """Recebe o código de uma ordem de produção e retorna seu status"""
    cursor.execute(f"SELECT status FROM ordem WHERE codigo = '{codigo}'")
    c = cursor.fetchone()
    return c[0]

In [36]:
def ConsultaOrdem(codigo):
    """Recupera o componente e a quantidade de produto a serem produzidos a partir de uma ordem cadastrada"""
    cursor.execute(f"SELECT quantidade, componente FROM ordem WHERE codigo = '{codigo}'")
    c = cursor.fetchone()
    quantidade = c[0]
    componente = c[1]
    return componente, quantidade

In [37]:
def ProduzirOrdem(maquina):
    """Inicia a produção de uma ordem"""
    codigo = input('Leia a ordem para produção: ')
    execucao = True
    
    status = Status(codigo)
    if not(status == 'Em fila' or status == 'Produzida em A'):
        return print(f'\nErro! A ordem já está em produção ou foi produzida. Status da ordem: {status}.')
    
    if maquina == 'A' and status != 'Produzida em A':
        sql = f"INSERT INTO produz(codigo, id_maquina, t_inicio) VALUES ('{codigo}','{maquina}',NOW())"
    elif maquina == 'B':
        if CheckFlow(codigo): 
            sql = f"INSERT INTO produz(codigo, id_maquina, t_inicio) VALUES ('{codigo}','{maquina}',NOW())"
        else:
            return print('\nErro! A ordem ainda não foi processada pela máquina A.')
    else:
        return print('\nErro! A ordem já foi produzida na máquina A.')
    
    try:
        cursor.execute(sql)
        ConfiguraMaquina(codigo, maquina)
        produto = ConsultaOrdem(codigo)
        print()
        print('Em produção: {} unidades de {} na máquina {}'.format(produto[1], produto[0], maquina))
        ConfigAtual(codigo, maquina)
    except mysql.connector.Error as err:
        print(f"Erro: {err.errno} - {err.msg}")
        execucao = False

    AtualizaStatusProcessando(codigo, maquina)
    return execucao

In [38]:
def AtualizaStatusProcessando(codigo, maquina):
    """Atualiza o status de processamento das ordens de acordo com a máquina em que são produzidas"""
    execucao = True
    
    if maquina == 'A':
        sql = f"UPDATE ordem SET status = 'Processando em A' WHERE codigo = '{codigo}'"
    elif maquina == 'B':
        sql = f"UPDATE ordem SET status = 'Processando em B' WHERE codigo = '{codigo}'"
    
    try:
        cursor.execute(sql)
    except mysql.connector.Error as err:
        print(f"Erro: {err.errno} - {err.msg}")
        execucao = False
        
    return execucao

In [39]:
def CheckFlow(codigo):
    """Analisa se o componente associado ao código já foi produzido na máquina A 
    para ser produzido na máquina B. Retorna verdadeiro ou falso."""
    codigos = []
    cursor.execute("SELECT codigo FROM produz WHERE id_maquina = 'A' AND t_fim IS NOT NULL")
    c = cursor.fetchall()
    
    for i in range(len(c)):
        codigos.append(c[i][0])
    
    return (codigo in codigos) 

In [40]:
def FinalizarOrdem():
    """Finaliza uma ordem de produção que esteja em andamento em qualquer uma das máquinas"""
    codigo = input('Leia a ordem para finalizar a produção: ')
    execucao = True
    
    status = Status(codigo)
    if status == 'Processando em A':
        sql = f"UPDATE produz SET t_fim = NOW() WHERE codigo = '{codigo}' AND id_maquina = 'A'"
        maquina = 'A'
    elif status == 'Processando em B':
        sql = f"UPDATE produz SET t_fim = NOW() WHERE codigo = '{codigo}' AND id_maquina = 'B'"
        maquina = 'B'
    else:
        return print('\nErro! A ordem não está em produção em nenhuma máquina.')
    
    try:
        cursor.execute(sql)
        produto = ConsultaOrdem(codigo)
        print()
        print('Produção finalizada: {} unidades de {} na máquina {}'.format(produto[1], produto[0], maquina))
    except mysql.connector.Error as err:
        print(f"Erro: {err.errno} - {err.msg}")
        execucao = False
        
    AtualizaStatusProduzida(codigo, maquina)
    return execucao

In [41]:
def AtualizaStatusProduzida(codigo, maquina):
    """Altera o status da ordem após o processamento"""
    execucao = True
    
    if maquina == 'A':
        sql = f"UPDATE ordem SET status = 'Produzida em A' WHERE codigo = '{codigo}'"
    elif maquina == 'B': 
        sql = f"UPDATE ordem SET status = 'Produzida' WHERE codigo = '{codigo}'"
    
    try:
        cursor.execute(sql)
    except mysql.connector.Error as err:
        print(f"Erro: {err.errno} - {err.msg}")
        execucao = False
        
    return execucao

### Configuração

In [42]:
def ConfiguraMaquina(codigo, maquina):
    """Configuração da máquina"""
    execucao = True
    
    sql = f"INSERT INTO configura VALUES ('{codigo}', '{maquina}')"
    
    try:
        cursor.execute(sql)
    except mysql.connector.Error as err:
        print(f"Erro: {err.errno} - {err.msg}")
        execucao = False
        
    return execucao

In [43]:
def ConfigAtual(codigo, maquina):
    """Consulta a configuração da máquina"""
    
    sql = f"SELECT configura.codigo, configura.id_maquina, ordem.temperatura, ordem.pressao, ordem.velocidade FROM maquina, ordem, configura WHERE configura.codigo = ordem.codigo AND ordem.codigo = '{codigo}' AND configura.id_maquina = maquina.id_maquina AND maquina.id_maquina = '{maquina}'"
    cursor.execute(sql)
    c = cursor.fetchall()
    temperatura = c[0][2]
    pressao = c[0][3]
    velocidade = c[0][4]
    return print(f'Configuração da máquina {maquina}: {temperatura}, {pressao}, {velocidade}')

### Relatórios

In [44]:
def TaxaProducao(maquina):
    """Relatório que fornece a quantidade produzida por segundo em uma máquina em determinado dia"""
    ano = int(input('Insira o ano da consulta: '))
    mes = int(input('Insira o número do mês da consulta: '))
    dia = int(input('Insira o dia da consulta: '))
    print()
    try:
        cursor.execute(f"SELECT sum(quantidade)/sum(tempo) AS \
                    taxa_producao_A FROM (SELECT ordem.quantidade,\
                    TIMESTAMPDIFF(second, t_inicio, t_fim) AS tempo FROM produz, \
                    ordem WHERE id_maquina = '{maquina}' AND DATE(t_inicio) = '{ano}-{mes}-{dia}' \
                    AND produz.codigo = ordem.codigo) as t")
        c = cursor.fetchone()
        c = float(c[0])
        r = f'Foram produzidas {c} unidades por segundo na máquina {maquina} em {dia}/{mes}/{ano}.'
    except:
        r = 'Não houve produção nessa data.'
    
    return print(r)

In [45]:
def LeadTime():
    """Apresenta a diferença entre o início da produção e a última ordem produzida na fábrica em determinado dia"""
    ano = int(input('Insira o ano da consulta: '))
    mes = int(input('Insira o número do mês da consulta: '))
    dia = int(input('Insira o dia da consulta: '))
    print()
    try:
        cursor.execute(f"SELECT TIMESTAMPDIFF(second, min(t_inicio), max(t_fim)) AS\
                lead_time FROM produz WHERE DATE(t_inicio) = '{ano}-{mes}-{dia}'")
        c = cursor.fetchone()
        c = float(c[0])
        r = f'O tempo de leadtime das ordens em {dia}/{mes}/{ano} foi de {c} segundos, ou {c/60:.2f} minuto(s).'
    except:
        r = 'Não houve produção nessa data.'
    
    return print(r)

### MES

In [ ]:
if __name__ == "__main__":
    cnx = ConectaBanco()
    if cnx.is_connected:
        print('Manufacturing Execution System')
        cursor = cnx.cursor()
        Menu()          
    else:
        print("Erro, banco de dados não conectado")
    cnx.close()

Manufacturing Execution System






------------------------------
       Tela inicial
------------------------------
1 - Cadastro de ordem
------------------------------
2 - Iniciar produção
------------------------------
3 - Finalizar produção
------------------------------
4 - Sair
------------------------------
5 - Status das ordens
------------------------------
6 - Relatórios
------------------------------
Entre com a opção desejada: 5

------------------------------
Ordens Cadastradas 

Código: 1000915092022   |   Status: Em fila
Código: 1002915092022   |   Status: Em fila
Código: 1012015092022   |   Status: Em fila
Código: 1052515092022   |   Status: Em fila
Código: 2004015092022   |   Status: Em fila
Código: 2010015092022   |   Status: Em fila
------------------------------






------------------------------
       Tela inicial
------------------------------
1 - Cadastro de ordem
------------------------------
2 - Iniciar produção
------------------------------
3 - Finaliza

Entre com a opção desejada: 3

-----------------------------------
Finalizar a produção de uma ordem
-----------------------------------
Leia a ordem para finalizar a produção: 2010015092022

Produção finalizada: 100 unidades de Produto 2 na máquina A






------------------------------
       Tela inicial
------------------------------
1 - Cadastro de ordem
------------------------------
2 - Iniciar produção
------------------------------
3 - Finalizar produção
------------------------------
4 - Sair
------------------------------
5 - Status das ordens
------------------------------
6 - Relatórios
------------------------------
Entre com a opção desejada: 2

-----------------------------------
Iniciar a produção de uma ordem
-----------------------------------
1 - Máquina A
2 - Máquina B
------------------------------
Insira o código da máquina: 1
------------------------------
Leia a ordem para produção: 1052515092022

Em produção: 525 unidades de Produto 1 na máquina A
Configuração

## Modelo físico MySQL

CREATE DATABASE manufatura;

USE manufatura;

SET SQL_SAFE_UPDATES = 0;

CREATE TABLE ordem (
	codigo varchar(20) primary key,
	componente varchar(15),
    quantidade int,
    status varchar(20),
    temperatura varchar(20),
    pressao varchar(20),
    velocidade varchar(20)
);

CREATE TABLE maquina (
	id_maquina char(1) primary key
);

CREATE TABLE produz (
	codigo varchar(20),
    id_maquina char(1),
    t_inicio datetime,
    t_fim datetime,
    FOREIGN KEY (codigo) REFERENCES ordem(codigo),
    FOREIGN KEY (id_maquina) REFERENCES maquina(id_maquina)
);	

CREATE TABLE configura (
	codigo varchar(20),
    id_maquina varchar(20),
    FOREIGN KEY (codigo) REFERENCES ordem(codigo),
    FOREIGN KEY (id_maquina) REFERENCES maquina(id_maquina)
);